In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [3]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

spacex_df=pd.read_csv(URL)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [7]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


In [8]:
# prompt: ## Task 1: Mark all launch sites on a map

# Add marker for each launch site
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    circle = folium.Circle(coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(row['Launch Site']))
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

In [9]:
# prompt: # Task 2: Mark the success/failed launches for each site on the map

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
marker_cluster = MarkerCluster()

# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color=assign_marker_color(record['class'])),
        popup=("Launch Site: " + str(record['Launch Site'])),
    )
    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)
site_map

In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [11]:
marker_cluster = MarkerCluster()


In [12]:
# prompt: # Apply a function to check the value of `class` column
# # If class=1, marker_color value will be green
# # If class=0, marker_color value will be red

# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color=assign_marker_color(record['class'])),
        popup=("Launch Site: " + str(record['Launch Site'])),
    )
    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)
site_map

In [13]:
# prompt: # TASK 3: Calculate the distances between a launch site to its proximities

# Import necessary libraries
from math import sin, cos, sqrt, atan2, radians

# Define a function to calculate the distance between two coordinates using the Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Example usage (replace with your actual coordinates)
launch_site_lat = 28.561857
launch_site_lon = -80.577366
proximity_lat = 28.5634
proximity_lon = -80.5768

distance = calculate_distance(launch_site_lat, launch_site_lon, proximity_lat, proximity_lon)
print(f"The distance between the launch site and the proximity is: {distance} km")


# Calculate distances for all launch sites and their proximities (replace with your actual data)
for index, row in launch_sites_df.iterrows():
    launch_site_lat = row['Lat']
    launch_site_lon = row['Long']
    # ... (Loop through proximity locations and calculate distances using the calculate_distance function)

The distance between the launch site and the proximity is: 0.18031492600714302 km


In [14]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [16]:
# prompt: # find coordinate of the closet coastline
# # e.g.,: Lat: 28.56367  Lon: -80.57163
# # distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Function to find the closest coastline coordinate
def find_closest_coastline(launch_site_lat, launch_site_lon, coastline_data):
    min_distance = float('inf')
    closest_coastline_coord = None

    for coastline_lat, coastline_lon in coastline_data:
        distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
        if distance < min_distance:
            min_distance = distance
            closest_coastline_coord = (coastline_lat, coastline_lon)

    return closest_coastline_coord, min_distance

# Example usage (replace with your actual coastline data)
# Assume coastline_data is a list of (latitude, longitude) tuples
coastline_data = [(28.56, -80.57), (28.57,-80.58), (28.55, -80.56)]  # Example coastline coordinates
launch_site_lat = 28.56367
launch_site_lon = -80.57163

closest_coord, distance_coastline = find_closest_coastline(launch_site_lat, launch_site_lon, coastline_data)

print(f"Closest coastline coordinate: {closest_coord}")
print(f"Distance to coastline: {distance_coastline} km")

Closest coastline coordinate: (28.56, -80.57)
Distance to coastline: 0.43817311396118086 km


In [17]:
# prompt: # Create and add a folium.Marker on your selected closest coastline point on the map
# # Display the distance between coastline point and launch site using the icon property
# # for example
# # distance_marker = folium.Marker(
# #    coordinate,
# #    icon=DivIcon(
# #        icon_size=(20,20),
# #        icon_anchor=(0,0),
# #        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
# #        )
# #    )

# Add a marker for the closest coastline point and display the distance
distance_marker = folium.Marker(
    closest_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )

site_map.add_child(distance_marker)
site_map

In [18]:
# prompt: # Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# # lines=folium.PolyLine(locations=coordinates, weight=1)
# site_map.add_child(lines)

# Assuming 'coordinates' is defined elsewhere in your code and contains the coastline coordinates.
# Also, assuming 'launch_site_coordinate' is defined and holds the launch site's coordinates.

import folium

# Example coordinates (replace with your actual data)
coordinates = [[28.56, -80.57], [28.57, -80.58], [28.55, -80.56]]
launch_site_coordinate = [28.56367, -80.57163]


lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [19]:
# prompt: # Create a marker with distance to a closest city, railway, highway, etc.
# # Draw a line between the marker to the launch site

# Assuming 'coordinates' is defined elsewhere in your code and contains the coastline coordinates.
# Also, assuming 'launch_site_coordinate' is defined and holds the launch site's coordinates.

# Example coordinates (replace with your actual data)
coordinates = [[28.56, -80.57], [28.57, -80.58], [28.55, -80.56]]
launch_site_coordinate = [28.56367, -80.57163]

# Create a polyline connecting the launch site to the coastline points
# We need to add launch site coordinate to the coordinates list in order to draw the line to it.
coordinates.append(launch_site_coordinate)
lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)
site_map